# Combine Multiple Embeddings

Combination of embeddings let's you create composite queries that depend on multiple input parameters.

In [1]:
%pip install superlinked==9.7.0

In [2]:
import pandas as pd

from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import String, Integer
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace
from superlinked.framework.dsl.query.param import Param

from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.space.number_space import NumberSpace, Mode
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.query.query import Query

pd.set_option("display.max_colwidth", 100)

In [3]:
@schema
class Paragraph:
    id: IdField
    body: String
    like_count: Integer


paragraph = Paragraph()

body_space = TextSimilaritySpace(
    text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2"
)
like_space = NumberSpace(
    number=paragraph.like_count, min_value=0, max_value=100, mode=Mode.MAXIMUM
)

## Combining vector spaces

Any number of spaces can be combined in an index.
Indices can be built on top of multiple spaces as simple as that. 

In [4]:
paragraph_index = Index([body_space, like_space])

In [5]:
source: InMemorySource = InMemorySource(paragraph)
executor = InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [6]:
source.put(
    [
        {
            "id": "paragraph-1",
            "body": "Glorious animals live in the wilderness.",
            "like_count": 10,
        },
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
            "like_count": 75,
        },
    ]
)

In [7]:
query = (
    Query(paragraph_index).find(paragraph).similar(body_space.text, Param("query_text"))
)

In [8]:
result = app.query(
    query,
    query_text="What makes the AI industry go forward?",
)

In [9]:
result.to_pandas()

,body,like_count,id
0,Growing computation power enables advancements in AI.,75,paragraph-2
1,Glorious animals live in the wilderness.,10,paragraph-1
